In [ ]:
import os
import bibtexparser 
import pandas as pd
import numpy as np
import yaml 

In [ ]:
# parametrsos globais
entrada = "input/"
saida = "output/"

# criando um lista com as colunas do escopo do projeto
col_types = [["author"] ,["title"],["keywords"],["abstract"],["year"],
                 ["type_publication","ENTRYTYPE"],["doi"]]

In [ ]:
# carregando os arquivos da pasta entrada
dir_list = os.listdir(entrada)
files = []

for item in dir_list:  
    
    if '.BIB' not in item.upper():
        continue
        
    files.append(item)
    

In [ ]:
## fazer a leitura de todos os arquivos
# criando um DF em branco para armazenar todos os arquivos
df_final = pd.DataFrame()

for file in files:
    
    ################################################################
    # iniciando a leitura do arquivo
    with open(entrada + file, encoding = "utf_8" ) as bibtex_file:
        
        bib_database = bibtexparser.load(bibtex_file)
        
        
        # convertendo o conteudo do arquivo para DF
        df = pd.DataFrame(bib_database.entries)

        #df.drop(0, inplace=True)
        del df["author"]

        ###############################################

    ################################################################################
    ## padronizando colunas
    # criando DF em branco para receber as colunas selecionadas do arquivo da vez
    new_df = pd.DataFrame()

    # interar lista de todos os tipos de colunas
    for col_type in col_types:

        # interar uma lista de um tipo de coluna apenas
        for col in col_type:
            
            # verificar se um item do tipo existe no DF
            if col in df.columns:
                # se existir criar um nova coluna no new_DF com o rótulo do primeiro item da lista do tipo da coluna
                new_df[col_type[0]] = df[col]
                break
            else:
                # se a coluna não existir então verificar se é a última coluna da lista do tipo
                # isso significa que não existe a coluna no DF, então criar um coluna com o rótulo do primeiro item 
                # com conteúdo em branco
                if col == col_type[-1]:
                    new_df[col_type[0]] = np.NaN    

    new_df["file"] = file
    ############################################
        
    # concatenando df 
    df_final = pd.concat([df_final, new_df]) 

df_final 

In [ ]:
# carregando a configuração do formato para exportação
with open('config/config.yaml', 'r') as config:
    formato = yaml.load(config, Loader=yaml.FullLoader)["export_to"]
    print(formato)

In [ ]:
arquivo_saida = saida + "arquivo_final." + formato

match formato:
  case "json":
    dados_arquivo = df_final.to_json(orient = "table")

    with open(arquivo_saida, "w") as arquivo:
      arquivo.write(dados_arquivo)

  case "csv":
    df_final.to_csv(arquivo_saida, sep = ";", header = True)
  
  case "yaml":
    dict_final = df_final.to_dict()

    with open(arquivo_saida, 'w') as file:
      yaml.dump(dict_final, file)